In [3]:
!pip install unidecode

In [15]:
import numpy as np
from math import gcd

# Función para encontrar el inverso modular
def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

# verifica si la matriz tiene inversa modular mod 26
def validate_key(matrix):
    det = int(round(np.linalg.det(matrix)))
    det_mod_26 = det % 26
    if gcd(det_mod_26, 26) != 1:
        return False
    return True

# Preprocesar el texto (convertir a números y manejar bloques)
def preprocess_text(text, block_size):
    text = text.replace(" ", "").upper()
    while len(text) % block_size != 0:
        text += 'X'  # Rellenar con 'X' si es necesario
    return text

# Convertir texto a números
def text_to_numbers(text):
    return [ord(char) - ord('A') for char in text]

# Convertir números a texto
def numbers_to_text(numbers):
    return ''.join([chr(num + ord('A')) for num in numbers])

# Convertir un string de 4 números en una matriz 2x2
def string_to_matrix(string):
    numbers = [int(digit) for digit in string.split()]
    return np.array(numbers).reshape(2, 2)

# Cifrar usando el cifrado de Hill
def hill_encrypt(plaintext, key):
    plaintext = preprocess_text(plaintext, 2)
    numbers = text_to_numbers(plaintext)
    key_matrix = np.array(key)

    if not validate_key(key_matrix):
        raise ValueError("La matriz no tiene inversa modular bajo módulo 26.")

    encrypted_numbers = []
    for i in range(0, len(numbers), 2):
        block = np.array(numbers[i:i+2]).reshape(-1, 1)
        encrypted_block = (np.dot(key_matrix, block) % 26).flatten()
        encrypted_numbers.extend(encrypted_block)

    return numbers_to_text(encrypted_numbers)

# Descifrar usando el cifrado de Hill
def hill_decrypt(ciphertext, key):
    numbers = text_to_numbers(ciphertext)
    key_matrix = np.array(key)

    if not validate_key(key_matrix):
        raise ValueError("La matriz no tiene inversa modular bajo módulo 26.")

    det = int(round(np.linalg.det(key_matrix)))
    det_mod_26 = det % 26
    inv_det = mod_inverse(det_mod_26, 26)

    adjugate = np.round(np.linalg.inv(key_matrix) * det).astype(int) % 26
    inverse_key_matrix = (inv_det * adjugate) % 26

    decrypted_numbers = []
    for i in range(0, len(numbers), 2):
        block = np.array(numbers[i:i+2]).reshape(-1, 1)
        decrypted_block = (np.dot(inverse_key_matrix, block) % 26).flatten()
        decrypted_numbers.extend(decrypted_block)

    return numbers_to_text(decrypted_numbers)

# Función para pedir una clave válida
def get_valid_key():
    while True:
        key_input = input("Introduce la clave como un string de 4 números separados por espacios: ")
        try:
            key = string_to_matrix(key_input)
            if validate_key(key):
                return key
            else:
                print("La matriz no tiene inversa modular bajo módulo 26. Intenta con otra clave.")
        except ValueError:
            print("Formato inválido. Asegúrate de ingresar 4 números separados por espacios.")

# Ejemplo de uso
if __name__ == "__main__":
    plaintext = input("Introduce el texto a cifrar: ")
    key = get_valid_key()

    encrypted_text = hill_encrypt(plaintext, key)
    print(f"Texto cifrado: {encrypted_text}")

    cyphertext = input("Introduce el texto a decifrar: ")
    cypherkey = get_valid_key()
    # Descifrar
    decrypted_text = hill_decrypt(cyphertext, cypherkey)
    print(f"Texto descifrado: {decrypted_text}")


Introduce el texto a cifrar: criptografia ejemplo
Introduce la clave como un string de 4 números separados por espacios: 24 3 1 5
La matriz no tiene inversa modular bajo módulo 26. Intenta con otra clave.
Introduce la clave como un string de 4 números separados por espacios: 5 8 17 3
Texto cifrado: QHEZZBKXOPOGORMAHCUV


KeyboardInterrupt: Interrupted by user